In [4]:
!pip install -q diffusers --upgrade
#!pip install invisible_watermark transformers accelerate safetensors
#!pip install -q transformers==4.37.2
!pip install -q gradio
!pip install -q git+https://github.com/openai/whisper.git

In [5]:
from diffusers import StableDiffusionPipeline
import torch
import whisper
import gradio as gr
from transformers import pipeline
DEVICE = 'cuda' if torch.cuda.is_available() else "cpu"

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

2024-02-27 10:08:05.972778: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-27 10:08:05.972900: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-27 10:08:06.097536: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
sentence = ""
model = whisper.load_model('medium', device = DEVICE)
model_id = 'stabilityai/stable-diffusion-2'
device = 'cuda'
pipe = StableDiffusionPipeline.from_pretrained(model_id, variant="fp16",torch_dtype=torch.float16)
pipe = pipe.to(device)
pipe.enable_attention_slicing()

100%|█████████████████████████████████████| 1.42G/1.42G [00:25<00:00, 60.2MiB/s]


model_index.json:   0%|          | 0.00/537 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

(…)ature_extractor/preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

tokenizer/tokenizer_config.json:   0%|          | 0.00/824 [00:00<?, ?B/s]

...

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

unet/config.json:   0%|          | 0.00/909 [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/1.73G [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [7]:
def speech_to_text(audio):
    res = model.transcribe(audio)
    sentence = res['text']
    return sentence

In [8]:
def text_to_image(text):
    promt = text + "4k, HDR, high resolution"
    image = pipe(prompt).images[0]
    return image

In [ ]:
import gradio as gr
import base64
import os

def process_inputs(audio_path):
    speech_to_text_output = speech_to_text(audio_path)
    text_to_image_output = text_to_image(speech_to_text_output)
    return speech_to_text_output, text_to_image_output

# Create the interface
iface = gr.Interface(
    fn=process_inputs,
    inputs=[
        gr.Audio(sources=["microphone"], type="filepath"),
    ],
    outputs=[
        gr.Textbox(label="Recognised speech"),
        gr.Image(label="Generated Image"),
    ],
    title="Speech  to Image",
    description="Upload an textual data via voice input."
)

# Launch the interface
iface.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860
Kaggle notebooks require sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Running on public URL: https://5388b6c327f0fc827d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
